<a href="https://colab.research.google.com/github/jcalbertin/mba/blob/master/Copy_of_Regress%C3%A3o_MLP_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr>
<h2><center>Disciplina : Redes Neurais Profundas</center></h2>
<h2><center>Regressão: Consumo de Combustível - TensorFlow & Keras</center></h2>
<center>Prof. Robson Fernandes</center>
<hr>

<p>Em um problema de regressão, o objetivo é prever as saídas (<em>outputs</em>) de um valor contínuo, como um preço ou probabilidade. Em contraste de problemas de classificação, onde temos o propósito de escolher uma classe em uma lista de classificações (por exemplo, se uma imagem contém uma maçã ou laranja, assim reconhecendo qual fruta é representada na imagem).</p>
<p>Este <em>notebook</em> usa a clássica base de dados <a href="https://archive.ics.uci.edu/ml/datasets/auto+mpg">Auto MPG</a> e constrói um modelo para prever a economia de combustíveis de automóveis do final dos anos 1970, início dos anos 1980. Para isso, forneceremos um modelo com descrição de vários automóveis desse período. Essa descrição inclui atributos como: cilindros, deslocamento, potência do motor, e peso.</p>
<p>Este exemplo usa a API <code>tf.keras</code>.

In [0]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## Importar a Base de Dados

<p>A base de dados está disponível  em <a href="https://archive.ics.uci.edu/ml/">UCI Machine Learning Repository</a>.</p>

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

Utilizando o pandas, impoorte os dados:

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

## Limpe os dados

Esta base contém alguns valores não conhecidos (unknown).

In [0]:
dataset.isna().sum()

Remova as linhas com esses valores não conhecidos.

In [0]:
dataset = dataset.dropna()

A coluna "Origin" é uma coluna categórica e não numérica. Logo converta para one-hot :

In [0]:
origin = dataset.pop('Origin')

In [0]:
## variaveis categoricas tem que ser convertidas para números
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

### Separando dados de treinamento e teste

Agora separe os dados em um conjunto de treinamento e outro teste.Iremos utilizar o de conjunto de teste no final da análise do model.


In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

## Análise Exploratória

Veja como está a distribuição de algumas colunas do conjunto de treinamento.

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

Veja a estatística geral

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

## Separe features de labels

Separe o valor alvo (labels), das features. Essa label é o valor no qual o modelo é treinado para prever.

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### Normalize os dados

Observe novamente o <b>train_stats</b> acima e note quão diferente são os intervalos de uma feature e outra.

Uma boa prática é normalizar as features que usam diferentes escalas e intervalos. Apesar do modelo poder convergir sem a normalização, isso torna o treinamento mais difícil, e torna o resultado do modelo dependente da escolha das unidades da entrada.

Observação: embora geramos intencionalmente essas estatísticas para os dados de treinamento, essas estatísticas serão usadas também para normalizar o conjunto de teste. Precisamos delinear o conjunto de teste na mesma distribuição que o modelo foi treinado.



In [0]:
def norm(x): ##x - cada registro - mormaliza
    return (x - train_stats['mean']) / train_stats['std'] # media / desvio padrao
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

print(normed_train_data)

Esse dado normalizado é o que usaremos para treinar o modelo.

Atenção: As estatísticas usadas para normalizar as entradas aqui (média e desvio padrão) precisa ser aplicada em qualquer outro dado que alimenta o modelo, junto com o código one-hot que fizemos anteriormente. Isso inclui o conjunto de teste e os dados que o modelo usará em produção.

## Contruindo Modelo

Vamos construir o modelo. Aqui usaremos o modelo <b>Sequential</b> com duas camadas densely connected, e a camada de saída que retorna um único valor contínuo. Os passos de construção do modelo são agrupados em uma função, build_model, já que criaremos um segundo modelo mais tarde.

In [0]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
    
    return model

In [0]:
model = build_model()

### Estrutura do modelo

Use o método .summary para exibir uma descrição simples do modelo.

In [0]:
model.summary()

Agora teste o modelo. Pegue um batch de de 10 exemplos do conjunto de treinamento e chame model.predict nestes.

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

## Treinando o Modelo

Treine o modelo com 1000 epochs, e grave a acurácia do treinamento e da validação em um objeto history

In [0]:
EPOCHS = 1000

logdir = "logs/"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2,
 callbacks=[tensorboard_callback])

### TensorBoard - Execute no Terminal

tensorboard --logdir=logs

Visualize o progresso do modelo de treinamento usando o estados armazenados no objeto history

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    plt.ylim([0,5])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    plt.ylim([0,20])
    plt.legend()
    plt.show()


plot_history(history)

Este grafo mostra as pequenas melhoras, ou mesmo a diminuição do validation error após 100 epochs. Vamos atualizar o model.fit para que pare automatixamente o treinamento quando o validation score não aumentar mais. Usaremos o EarlyStopping callback que testa a condição do treinamento a cada epoch. Se um grupo de epochs decorre sem mostrar melhoras, o treinamento irá parar automaticamente.

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop])

plot_history(history)

O gráfico mostra que no conjunto de validação, a média de erro é próximo de +/- 2MPG.

Vamos ver quão bem o modelo generaliza usando o conjunto de <b>teste</b>, que não usamos para treinar o modelo. Isso diz quão bem podemos esperar que o modelo se saia quando usarmos na vida real.

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### Realizando Inferência

Finalmente, prevejamos os valores MPG usando o conjunto de teste.

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

Parece que o nosso modelo prediz razoavelmente bem. Vamos dar uma olhada na distribuição dos erros.

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

Embora não seja uma gaussiana perfeita, o distribuição dos erros possuem um comportamente bem próximo da normalidade, conforme análise gráfica.